In [ ]:
# Exemplos de:
# Ponte/Decorador/Compósito/Abstract-Factory/Mediator/State

````
Para acompanhar a estrutura das classes desenvolvidas (e saber o que está acontecendo nos testes), leia este arquivo junto com o "dp-exemplos.pdf" disponível neste mesmo diretório
````

# Exemplo 1: Ponte

In [40]:
# Exemplo 1: Ponte

import abc


# OBS: Um "Node" é qualquer tipo de objeto passível de hash


class TreeImplementation(abc.ABC):
    @abc.abstractmethod
    def add(self, parent, child):
        """
        Adds child to parent in the tree
        :param parent: A node in the tree
        :param child: A node not in the tree
        """
        pass

    @abc.abstractmethod
    def rm(self, node):
        """
        Removes 'node' from the tree along with its entire subtree
        :param node: A node in the tree
        """
        pass

    @abc.abstractmethod
    def root(self):
        """
        Gets the root
        :return: A node (or None if tree is empty)
        """
        pass

    @abc.abstractmethod
    def children(self, node):
        """
        Get children of node
        :param node: A node in the tree
        :return: list with children (may be empty list)
        """
        pass

    @abc.abstractmethod
    def parent(self, node):
        """
        Gets parent of node
        :param node: A node in the tree
        :return: A node in the tree, or None if 'node' is root
        """
        pass


class Tree(object):
    def __init__(self, implementation):
        """
        :param implementation: Subclass of TreeImplementation containing implementation 
        """
        self._impl = implementation

    def add(self, parent, child):
        return self._impl.add(parent, child)

    def rm(self, node):
        return self._impl.rm(node)

    def root(self):
        return self._impl.root()

    def children(self, node):
        return self._impl.children(node)

    def parent(self, node):
        return self._impl.parent(node)


class TreeWithDicts(TreeImplementation):
    """
    'node' refers to internal implementation
    'value' refers to what clients of this class store in the tree
    A 'value' must be unique across the tree
    """

    def __init__(self):
        self.__nodes = None  # Each node is { value: any , children: list }

    def __find_by_value(self, value, start_node):
        """
        :param value: A value (user-defined) 
        :param start_node: A node (implementation internal)
        :return: (node, parent_node)
        """
        if value == start_node['value']:
            return start_node, 'caller'

        for child in start_node['children']:
            node, parent = self.__find_by_value(value, child)
            if parent == 'caller':
                parent = start_node
            if node is not None:
                return node, parent

        return None, None

    def add(self, parent, child):
        # Insert root
        if parent is None and self.__nodes is None:
            self.__nodes = {'value': child, 'children': []}
        # Insert child
        elif parent is not None and self.__nodes is not None:
            parent_node, _ = self.__find_by_value(parent, self.__nodes)
            if parent_node is not None:
                parent_node['children'].append({'value': child, 'children': []})
                return

    def rm(self, node):
        if self.__nodes is None:
            return

        internal_node, internal_parent = self.__find_by_value(node, self.__nodes)
        if internal_parent == 'caller':
            internal_parent = None

        if internal_node is None:
            return

        if internal_parent is None:
            self.__nodes = {}
        else:
            internal_parent['children'].remove(internal_node)

    def root(self):
        return self.__nodes['value']

    def children(self, node):
        if self.__nodes is None:
            return None

        internal_node, _ = self.__find_by_value(node, self.__nodes)

        if internal_node is None:
            return None

        return [child['value'] for child in internal_node['children']]

    def parent(self, node):
        if self.__nodes is None:
            raise Exception('Tried to find parent of a node in an empty tree')

        internal_node, internal_parent = self.__find_by_value(node, self.__nodes)
        if internal_parent == 'caller':
            internal_parent = None

        if internal_node is None:
            return None

        if internal_parent is None:
            return None

        return internal_parent['value']


class Node(object):
    def __init__(self, label):
        self.label = label


class MathTree(Tree):
    """
    Expects to work with Node instances
    """
    
    def __init__(self):
        super().__init__(TreeWithDicts())

    def __evaluate(self, node: Node):
        if node.label == '+':
            children = self.children(node)
            return self.__evaluate(children[0]) + self.__evaluate(children[1])
        elif node.label == '-':
            children = self.children(node)
            return self.__evaluate(children[0]) - self.__evaluate(children[1])
        elif node.label == '/':
            children = self.children(node)
            return self.__evaluate(children[0]) / self.__evaluate(children[1])
        elif node.label == '*':
            children = self.children(node)
            return self.__evaluate(children[0]) * self.__evaluate(children[1])
        # Expects number
        else:
            return node.label

    def evaluate(self):
        return self.__evaluate(self.root())


# Teste do exemplo 1

In [43]:
# We build the expression "1 + (2 - (8 / 2)) * 3"
mult = Node('*')
div = Node('/')
minus = Node('-')
plus = Node('+')
three = Node(3)
right2 = Node(2)
eight = Node(8)
left2 = Node(2)
one = Node(1)

a = MathTree()
a.add(None, plus)
a.add(plus, one)
a.add(plus, mult)
a.add(mult, minus)
a.add(mult, three)
a.add(minus, left2)
a.add(minus, div)
a.add(div, eight)
a.add(div, right2)
a.evaluate()  # Faça as contas ! Dá -5 mesmo...


-5.0

# Exemplo 2: Composição

In [47]:
class TreeWithComposition(TreeImplementation):
    """
    Nodes must be instances of TreeWithComposition itself !
    
    The "root" of a TreeWithComposition is the instance itself
    
    The tree cannot be empty
    """
    
    def __init__(self):
        super().__init__()
        self.__children = []

    def add(self, parent, child):
        parent.__children.append(child)
    
    def __recurse_rm(self, node):
        for child in self.__children:
            if child == node:
                self.__children.remove(child)
                return True
            if child.__recurse_rm(node) is True:
                return

    def rm(self, node):
        self.__recurse_rm(node)

    def root(self):
        return self

    def children(self, node):
        return self.__children
    
    def __recurse_parent(self, node):
        if node in self.__children:
            return self
        for child in self.__children:
            parent = child.__recurse_parent(node)
            if parent:
                return parent
        return None

    def parent(self, node):
        return self.__recurse_parent(node)

# Exemplo 3: Mediador + Fábrica Abstrata + Estado + Decorador

In [370]:
# Só implemento algumas das classes mencionadas no pdf, porque as outras são
# só ilustrativas do que é possível fazer

from threading import Timer
import abc
from weakref import WeakSet


class ChatRoom(abc.ABC):
    @abc.abstractmethod
    def connect(self):
        """
        Connects the channel. After connected, messages can be sent
        """
        pass

    @abc.abstractmethod
    def disconnect(self):
        """
        Disconnects. After disconnected, messages can no longer be sent or received        
        """
        pass

    @abc.abstractmethod
    def send(self, message):
        """
        Sends message to all connected peers
        :param message: string to send
        """
        pass

    @abc.abstractmethod
    def onreceive(self, callback):
        """
        Sets a function "(message: string) => None" to be executed when a message
        is received
        :param callback: Function accepting 1 arg (string message) and returning nothing 
        """
        pass

    @abc.abstractmethod
    def ondisconnect(self, callback):
        """
        Sets a function "() => None" to be executed when disconnect is complete
        :param callback: Function accepting 0 arg and returning nothing 
        """
        pass

    @abc.abstractmethod
    def onconnect(self, callback):
        """
        Sets a function "() => None" to be executed when connect is complete
        :param callback: Function accepting 0 arg and returning nothing 
        """
        pass
    
    @abc.abstractmethod
    def is_connected(self):
        """
        True if channel is connected. False otherwise
        """
        pass


class State(abc.ABC):
    def __init__(self, room, onconnect, ondisconnect, onreceive):
        self.room = room
        self.onconnect = onconnect
        self.ondisconnect = ondisconnect
        self.onreceive = onreceive

    @abc.abstractmethod
    def send(self, message):
        pass

    @abc.abstractmethod
    def connect(self):
        pass

    @abc.abstractmethod
    def disconnect(self):
        pass

    @abc.abstractmethod
    def receive(self, message):
        pass
    
    @abc.abstractmethod
    def is_connected(self):
        pass


class Connected(State):
    def __init__(self, *args):
        super().__init__(*args)
        self.room.connected_clients.add(self.room)

    def send(self, message):
        for room in self.room.connected_clients:
            if room != self.room:
                room.receive(message)

    def connect(self):
        pass

    def disconnect(self):
        self.room.connected_clients.remove(self.room)
        self.room.set_state(Disconnected(
            self.room, self.onconnect, self.ondisconnect, self.onreceive
        ))
        self.ondisconnect()

    def receive(self, message):
        self.onreceive(message)
        
    def is_connected(self):
        return True


class Connecting(State):
    def __init__(self, *args):
        super().__init__(*args)
        Timer(3, lambda: self.room.set_state(Connected(
            self.room, self.onconnect, self.ondisconnect, self.onreceive
        ))).start()

    def send(self, message):
        raise RuntimeError('Tried to send a message while channel was in connecting state')

    def connect(self):
        pass

    def disconnect(self):
        self.room.set_state(Disconnected(
            self.room, self.onconnect, self.ondisconnect, self.onreceive
        ))
        self.ondisconnect()

    def receive(self, message):
        pass
    
    def is_connected(self):
        return False


class Disconnected(State):
    def send(self, message):
        raise RuntimeError('Tried to send a message while channel was in disconnected state')

    def connect(self):
        self.room.set_state(Connecting(
            self.room, self.onconnect, self.ondisconnect, self.onreceive
        ))
        self.onconnect()

    def disconnect(self):
        pass

    def receive(self, message):
        pass
    
    def is_connected(self):
        return False


class RoomInMemory(ChatRoom):
    connected_clients = WeakSet()

    def __init__(self):
        self.__state = Disconnected(
            self, lambda: None, lambda: None, lambda msg: None
        )

    def connect(self):
        self.__state.connect()

    def disconnect(self):
        self.__state.disconnect()

    def send(self, message):
        self.__state.send(message)

    def onreceive(self, callback):
        self.__state.onreceive = callback

    def ondisconnect(self, callback):
        self.__state.ondisconnect = callback

    def onconnect(self, callback):
        self.__state.onconnect = callback
        
    def is_connected(self):
        return self.__state.is_connected()
        
    def set_state(self, state):
        self.__state = state

    def receive(self, message):
        self.__state.receive(message)


class RoomFactory(abc.ABC):
    
    @abc.abstractmethod
    def named(self, name):
        """
        Returns a chat room
        :param name: name of the channel
        """
        pass


class InMemoryFactory(RoomFactory):
    
    def named(self, name):
        return RoomInMemory()


## Testes parciais do exemplo 3

In [299]:
client1 = RoomInMemory()

In [300]:
client2 = RoomInMemory()

In [301]:
client1.onreceive(lambda msg: print('client1 receives: ' + msg))

In [302]:
client2.onreceive(lambda msg: print('client2 receives: ' + msg))

In [303]:
client1.connect()

In [304]:
client2.connect()

In [307]:
client2.send('message from client2')

client1 receives: message from client2


In [308]:

client1.send('client 1 saying nothing')

client2 receives: client 1 saying nothing


In [309]:
client1.disconnect()

In [310]:
client2.send('I know client1 disconnected, '
             'so he cannot receive this (therefore no prints)')

In [311]:
client3 = RoomInMemory()
client3.connect()

In [312]:
client3.onreceive(lambda msg: print('client 3 receiving: ' + msg))

In [313]:
client1.connect()

In [316]:
client2.send('I know I have 2 receivers')

client1 receives: I know I have 2 receivers
client 3 receiving: I know I have 2 receivers


## Decorador do exemplo 3

In [373]:
class KeepAlive(ChatRoom):
    
    def is_connected(self):
        return True

    def __init__(self, room):
        """
        Maintains a chat room always alive. As consequence,
        clients of this class will never receive the 'ondisconnect' and
        'onconnect' hooks, neither can the client try to disconnect 
        from the room
        :param room: An instance of ChatRoom to be kept alive 
        """
        self.__queued_msgs = []
        self.__room = room
        self.__room.onconnect(self.__onconnect)
        self.__room.ondisconnect(self.__ondisconnect)
        self.__room.connect()
    
    def __onconnect(self):
        for msg in self.__queued_msgs:
            self.send(msg)
        self.__queued_msgs = []
        
    def __ondisconnect(self):
        self.__room.connect()

    def connect(self):
        pass

    def disconnect(self):
        pass

    def send(self, message):
        if self.is_connected():
            self.__room.send(message)
        else:
            self.__queued_msgs.append(message)

    def onreceive(self, callback):
        self.__room.onreceive(callback)

    def ondisconnect(self, callback):
        pass

    def onconnect(self, callback):
        pass

## Últimos testes do exemplo 3

In [374]:
internal = InMemoryFactory().named('my_channel')
client = KeepAlive(internal)

In [396]:
client.is_connected()

True

In [376]:
internal.is_connected()

True

In [377]:
internal.disconnect()

In [383]:
# Internal room is still alive !
internal.is_connected()

True

In [384]:
client2 = InMemoryFactory().named('my_channel')

In [387]:
client2.connect()

In [398]:
client.onreceive(lambda msg: print('client1 receives:' + msg))

In [399]:
client2.send('Hello')

client1 receives:Hello
